# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [78]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [79]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [80]:
customer_df = pd.DataFrame(valuesA, columns=valuesA_cols)
orders_df = pd.DataFrame(valuesB, columns=valuesB_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set? 

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [83]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [84]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [85]:
joined_df.groupby('name').agg({"price": ["sum"]}) 

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [86]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
| bob|             4.55|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [ ]:
# save to the filesystem to prevent another load
! wget curl -oP https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

In [39]:
import pandas as pd
nyc_df = pd.read_csv('out.csv')

/home/ubuntu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

1706756


We'll take a random sample at 20% of the original data

In [89]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [104]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1', 
                       'BOROUGH', 'VEHICLE TYPE CODE 1', 
                       'NUMBER OF PERSONS INJURED']]

In [105]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
0,04/10/2019,Unspecified,QUEENS,Station Wagon/Sport Utility Vehicle,1.0
1,05/01/2019,Driver Inattention/Distraction,BROOKLYN,Sedan,0.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [124]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out 
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf.schema)

StructType(List(StructField(CRASH DATE,StringType,true),StructField(CONTRIBUTING FACTOR VEHICLE 1,StringType,true),StructField(BOROUGH,StringType,true),StructField(VEHICLE TYPE CODE 1,StringType,true),StructField(NUMBER OF PERSONS INJURED,StringType,true)))


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [125]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

CRASH DATE
05/02/2019    49
04/12/2019    45
04/16/2019    43
04/23/2019    43
04/15/2019    43
Name: CRASH DATE, dtype: int64

In [130]:
### Spark?



+----------+-----+
|CRASH DATE|count|
+----------+-----+
|05/02/2019|   49|
|04/12/2019|   45|
|04/23/2019|   43|
|04/16/2019|   43|
|04/15/2019|   43|
+----------+-----+
only showing top 5 rows



# Question 2

> _Where do most crashes occur?_

In [55]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         219
QUEENS           180
MANHATTAN        149
BRONX             88
STATEN ISLAND     17
Name: BOROUGH, dtype: int64

In [ ]:
## Spark?



 # Question 3
 
 > What is the most common cause of accident in 'QUEENS'

In [61]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    55
Unspecified                       40
Failure to Yield Right-of-Way     23
Passing or Lane Usage Improper    10
Passing Too Closely               10
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [ ]:
## Spark?





# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [68]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

VEHICLE TYPE CODE 1  BOROUGH 
Taxi                 QUEENS      1.6
Bike                 BRONX       1.0
                     BROOKLYN    1.0
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [1]:
## Spark?